## Train models

### Asif Hamid bhat 

This script is a template for training various models.

In [1]:
### clear ram
# import torch
# with torch.no_grad():
#     torch.cuda.empty_cache()
# %reset

In [1]:
import os
import matplotlib.pyplot as plt
import sys
import torch
import numpy as np
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path) 
from Autoencoder import *
from utils import *
import ResNet as net
# import lstmrnn

In [2]:
# adjustables
k = list(range(11))           
dt = 0.01  
model_prefix='KS'
system = 'KS'         # system name: ""FHN","KS
noise = 0.0                    
hidden_size=8
lr = 1e-3                     # learning rate
max_epoch = 10000            # the maximum training epoch

In [3]:
# paths
data_dir = os.path.join('../data/', system)
model_dir = os.path.join('../models/', system)
path_to_result=os.path.join('../results/', system)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# global const
n_forward = 5

### LOAD DATA

### LOAD DATA

In [4]:
train_data = torch.load(data_dir + "/train_data.pt")
val_data = torch.load(data_dir + "/val_data.pt")
test_data = torch.load(data_dir + "/test_data.pt")
train_data=train_data[:10,:,:]

n=train_data.shape[2]
n_train = train_data.shape[0]

train_data.shape,val_data.shape,test_data.shape

(torch.Size([10, 5121, 120]),
 torch.Size([5, 5121, 120]),
 torch.Size([5, 5121, 120]))

In [5]:
train_data=torch.tensor(train_data)
train_data = train_data.to(torch.float32).to(device)
sets=train_data.shape[0]
dataloader=torch.utils.data.DataLoader(train_data, batch_size=32) ### 32 for FHN 

#### Train autoencoder

In [8]:
# define model and train
######Define the autoencoder class
input_size=train_data.shape[2]
L1=120  ### 100,100,100 for fhn and 120,120,100 for KS
L2=120
L3=100
modelAE = AE(L1,L2,L3,input_size, hidden_size).to(device)
num_epochs = 5000 # 5,000 for both FHN,KS
learning_rate = 1e-3
aemodel='/AE_{}.pt'.format(hidden_size)
save_path=model_dir+aemodel
train(modelAE, dataloader, num_epochs=num_epochs, learning_rate=1e-3,model_path=save_path)

In [10]:
####Load FHN AE BEST models
modelAE.load_state_dict(torch.load(save_path))

In [17]:
#Data for latent dimension
z_train=modelAE.encoder(train_data)
z_valid=modelAE.encoder(torch.tensor(val_data[:,:,:]).to(torch.float32).to(device))
z_test=modelAE.encoder(torch.tensor(test_data[:,:,:]).to(torch.float32).to(device))
z_train.shape,z_valid.shape,z_test.shape

(torch.Size([10, 5121, 10]),
 torch.Size([5, 5121, 10]),
 torch.Size([5, 5121, 10]))

In [18]:
# data.shape

#### Set wise training HIts

In [21]:
z_train1=z_train
data=z_train1#.reshape(z_train1.shape[0]*z_train1.shape[1],z_train1.shape[2])
data=data.cpu().detach().numpy()
n_steps = data.shape[1] - 1
data.shape

(10, 5121, 10)

In [26]:
### one set training 
# dummy parameter
data=z_train1[:10,:,:]#.reshape(z_train1.shape[0]*z_train1.shape[1],z_train1.shape[2]).... can also use reshape option
sets=z_train1.shape[0]
data=data.cpu().detach().numpy()
models = list()
L1=128
L2=256
L3=512
L4=1024
L5=2048
z=hidden_size
dt = 1  
n_forward=5
step_sizes=[1,2,4,8,16,32,64,128,256,512,1024]
# training
max_epoch=20000
start=time.time()
n_steps = data.shape[1] - 1  # number of forward steps
for step_size in step_sizes:
    resdata = np.empty((0, step_size * n_forward + 1, z_train1.shape[2]))
    for j in range(sets):
        data=z_train1[j,:,:].cpu().detach().numpy()
        m = int(np.ceil(n_steps/(step_size*n_forward)))
        pdata = np.zeros((m, step_size*n_forward+1, data.shape[1]))
        for i in range(m):
            start_idx = i*step_size*n_forward
            end_idx = start_idx + step_size*n_forward + 1
            tmp = data[start_idx:end_idx, :]
            pdata[i, :tmp.shape[0], :] = tmp
        resdata = np.concatenate((resdata, pdata), axis=0)
    pdata=resdata
    dataset = net.DataSet(pdata, pdata, data[np.newaxis, :], dt, step_size, n_forward)
    model_name = 'modelKS_D{}_h{}.pt'.format(step_size,hidden_size)
    print('MODEL: '+model_prefix+'_D{}_h{}'.format(step_size,hidden_size))
    model = net.ResNet(arch=[z,L4,L4,L4,z], dt=dt, step_size=step_size)
    model.train_net(dataset, max_epoch=max_epoch, batch_size=32, lr=1e-3, model_path=os.path.join(model_dir, model_name))
    models.append(model)
end=time.time()
train_time=end-start
print(train_time)
print('# of params: ', sum([sum(p.numel() for p in model.parameters() if p.requires_grad) for model in models]))

MODEL: KS_D1_h10
epoch 1000, training loss 0.0007058691699057817, validation loss 0.0008411856251768768
(--> new model saved @ epoch 1000)
epoch 2000, training loss 0.0004134595219511539, validation loss 0.0004330627270974219
(--> new model saved @ epoch 2000)
epoch 3000, training loss 0.0004702173755504191, validation loss 0.00040564732626080513
(--> new model saved @ epoch 3000)
epoch 4000, training loss 0.00032227145857177675, validation loss 0.0003826737229246646
(--> new model saved @ epoch 4000)
epoch 5000, training loss 0.0003899821313098073, validation loss 0.00038749989471398294
epoch 6000, training loss 0.0002578617713879794, validation loss 0.0003761422703973949
(--> new model saved @ epoch 6000)
epoch 7000, training loss 0.0003352499916218221, validation loss 0.0003616265021264553
(--> new model saved @ epoch 7000)
epoch 8000, training loss 0.00034778594272211194, validation loss 0.00035482741077430546
(--> new model saved @ epoch 8000)
epoch 9000, training loss 0.000398415